# P4: URS with Disruptions - Multiagent Pattern


<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together.

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.

**P4: URS with Disruptions** - Planning and scheduling with traffic delays and dynamic changes to route feasibility.


In [ ]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"


In [ ]:
import sys
print("Python version:", sys.version)


In [ ]:
%pip install openai python-dotenv graphviz colorama


In [ ]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")


In [ ]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)


## The Agent Class


First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.


In [ ]:
import sys
import os
os.environ["OPENAI_API_KEY"]=""
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)


In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")


You can also associate tools with the agent. Let's create a tool for writing some string into a file.


In [ ]:
from src.tool_agent.tool import tool

@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")


## The Crew for P4: URS with Disruptions


In [ ]:
from src.multi_agent.crew import Crew

with Crew() as crew:
    # ---- Data Collection Agent ---- #
    DC_Agent = Agent(
        name="Data Collection Agent",
        backstory="You collect traffic data, road closure updates, and estimated travel times between locations, including real-time disruption information.",
        task_description="Retrieve traffic conditions, road closures, disruptions, and estimated travel durations for all routes involved in passenger transport.",
        task_expected_output="Structured travel time data with disruption information: 1) CityMap with locations and roads. 2) Ride Requests with time windows. 3) Vehicles with capacity. 4) Current disruptions (traffic delays, road closures)."
    )
    
    # ---- Route Planning Agent ---- #
    RP_Agent = Agent(
        name="Route Planning Agent",
        backstory="You determine optimal routes for vehicles to minimize total travel time while ensuring all passengers arrive on time, considering current disruptions.",
        task_description="Use traffic data and disruption information to compute the best routes for each vehicle, avoiding blocked routes and accounting for delays.",
        task_expected_output="Optimized vehicle assignments and travel routes that avoid disruptions and minimize delays."
    )
    
    # ---- Traffic Adjustment Agent ---- #
    TA_Agent = Agent(
        name="Traffic Adjustment Agent",
        backstory="You monitor live traffic updates and adjust vehicle routes dynamically in case of delays and disruptions.",
        task_description="Recompute routes in real time when disruptions occur (traffic delays, road closures), ensuring minimal delays and alternative route planning.",
        task_expected_output="Updated travel plans for affected vehicles with alternative routes and adjusted schedules."
    )
    
    # ---- Vehicle Dispatch Agent ---- #
    VD_Agent = Agent(
        name="Vehicle Dispatch Agent",
        backstory="You assign passengers to vehicles and ensure each vehicle follows the optimal planned route, adapting to disruptions.",
        task_description="Assign passengers to vehicles based on capacity constraints and route efficiency, considering disruption impacts.",
        task_expected_output="Vehicle assignment list and dispatch schedule adapted to current disruptions."
    )
    
    # ---- Monitoring & Alert Agent ---- #
    MA_Agent = Agent(
        name="Monitoring & Alert Agent",
        backstory="You track vehicle movements and notify if there are risks of missing passenger deadlines due to disruptions.",
        task_description="Send alerts for potential delays and recommend contingency plans when disruptions occur.",
        task_expected_output="Timely notifications for alternative route adjustments or emergency responses due to disruptions."
    )

    Writer_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './p4_output.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )
    
    # ---- Define Dependencies ---- #
    DC_Agent >> RP_Agent >> TA_Agent >> VD_Agent >> MA_Agent >> Writer_agent


In [ ]:
crew.run()
